# Day 10

## Part 1

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

- `|` is a vertical pipe connecting north and south.
- `-` is a horizontal pipe connecting east and west.
- `L` is a 90-degree bend connecting north and east.
- `J` is a 90-degree bend connecting north and west.
- `7` is a 90-degree bend connecting south and west.
- `F` is a 90-degree bend connecting south and east.
- `.` is ground; there is no pipe in this tile.

S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

```
.....
.F-7.
.|.|.
.L-J.
.....
```
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

```
.....
.S-7.
.|.|.
.L-J.
.....
```
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:
```
.....
.S-7.
.|.|.
.L-J.
.....
```
You can count the distance each tile in the loop is from the starting point like this:

```
.....
.012.
.1.3.
.234.
.....
```
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```
Here are the distances for each tile on that loop:

```
..45.
.236.
01.78
14567
23...
```
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [75]:
from typing import Optional


class MapPosition:
    can_connect_north = False
    can_connect_south = False
    can_connect_west = False
    can_connect_east = False

    def __init__(self, lat: int, lon: int):
        self.lat = lat
        self.lon = lon

    def __eq__(self, other: "MapPosition") -> bool:
        if other is None:
            return False
        return self.coords == other.coords

    @property
    def coords(self) -> tuple[int, int]:
        return (self.lat, self.lon)

    def __repr__(self):
        return "░"


class WEPipe(MapPosition):
    can_connect_west = True
    can_connect_east = True

    def __repr__(self):
        return "═"


class NSPipe(MapPosition):
    can_connect_north = True
    can_connect_south = True

    def __repr__(self):
        return "║"


class NWPipe(MapPosition):
    can_connect_north = True
    can_connect_west = True

    def __repr__(self):
        return "╝"


class NEPipe(MapPosition):
    can_connect_north = True
    can_connect_east = True

    def __repr__(self):
        return "╚"


class SWPipe(MapPosition):
    can_connect_south = True
    can_connect_west = True

    def __repr__(self):
        return "╗"


class SEPipe(MapPosition):
    can_connect_south = True
    can_connect_east = True

    def __repr__(self):
        return "╔"


class StartPosition(MapPosition):
    can_connect_north = True
    can_connect_south = True
    can_connect_east = True
    can_connect_west = True

    def __repr__(self):
        return "O"


def convert_to_position(pos: str, lat: int, lon: int) -> MapPosition:
    if pos == ".":
        obj = MapPosition
    elif pos == "-":
        obj = WEPipe
    elif pos == "J":
        obj = NWPipe
    elif pos == "7":
        obj = SWPipe
    elif pos == "L":
        obj = NEPipe
    elif pos == "F":
        obj = SEPipe
    elif pos == "|":
        obj = NSPipe
    elif pos == "S":
        obj = StartPosition
    else:
        raise ValueError(f"Unknown position: {pos}")
    return obj(lat, lon)


class PipeMap:
    def __init__(self, map: str):
        self.data = self.generate_map(map)

    def generate_map(self, map: str) -> list[list[MapPosition]]:
        new_rows = []
        map_lines = map.splitlines()
        for lat, row in enumerate(map_lines):
            new_row = []
            for lon, pos in enumerate(row):
                obj = convert_to_position(pos, lat, lon)
                new_row.append(obj)
            new_rows.append(new_row)
        return new_rows


class MapPositionRunner:
    def __init__(self, map_str: str):
        self.pipe_map = PipeMap(map_str)
        self.traveled_positions = []

    @property
    def map_data(self) -> list[list[MapPosition]]:
        return self.pipe_map.data

    @property
    def starting_position(self) -> StartPosition:
        for row in self.map_data:
            for pos in row:
                if isinstance(pos, StartPosition):
                    return pos

    def __repr__(self) -> str:
        map_repr = ""
        for row in self.map_data:
            for pos in row:
                map_repr += repr(pos)
            map_repr += "\n"
        return map_repr

    @property
    def traveled_loop(self) -> str:
        map_repr = ""
        for row in self.map_data:
            for pos in row:
                pos_repr = pos
                if pos in self.traveled_positions:
                    # highlight red
                    pos_repr = f"\033[91m{pos_repr}\033[0m"
                map_repr += f"{pos_repr}"
            map_repr += "\n"
        return map_repr

    def north_position(self, from_position: MapPosition) -> Optional[MapPosition]:
        if from_position.lat != 0 and from_position.can_connect_north:
            to_position: MapPosition = self.map_data[from_position.lat - 1][from_position.lon]
            if to_position.can_connect_south:
                return to_position
        return None

    def south_position(self, from_position: MapPosition) -> Optional[MapPosition]:
        if from_position.lat != len(self.map_data) - 1 and from_position.can_connect_south:
            to_position: MapPosition = self.map_data[from_position.lat + 1][from_position.lon]
            if to_position.can_connect_north:
                return to_position
        return None

    def west_position(self, from_position: MapPosition) -> Optional[MapPosition]:
        if from_position.lon != 0 and from_position.can_connect_west:
            to_position: MapPosition = self.map_data[from_position.lat][from_position.lon - 1]
            if to_position.can_connect_east:
                return to_position
        return None

    def east_position(self, from_position: MapPosition) -> Optional[MapPosition]:
        if (
            from_position.lon != len(self.map_data[from_position.lat]) - 1
            and from_position.can_connect_east
        ):
            to_position: MapPosition = self.map_data[from_position.lat][from_position.lon + 1]
            if to_position.can_connect_west:
                return to_position
        return None

    def available_moves(self, from_position: MapPosition):
        moves = []
        for direction in ("north", "south", "east", "west"):
            move = getattr(self, f"{direction}_position")(from_position)
            if move:
                moves.append(move)
        return moves

    def travel_loop(self) -> int:
        self.traveled_positions = []

        n_moves = 0
        last_position = None
        current_position = self.starting_position
        while True:
            self.traveled_positions.append(current_position)
            moves = [
                m
                for m in self.available_moves(current_position)
                if m != last_position and m != self.starting_position
            ]
            if not moves:
                break

            last_position = current_position
            current_position = moves[0]
            n_moves += 1

        return (n_moves // 2) + 1


test_inputs_a = """.....
.S-7.
.|.|.
.L-J.
....."""

test_runner_a = MapPositionRunner(test_inputs_a)
print(test_runner_a)
print(test_runner_a.starting_position.coords)
test_distances_a = test_runner_a.travel_loop()
print(test_runner_a.traveled_loop)
test_distances_a

░░░░░
░O═╗░
░║░║░
░╚═╝░
░░░░░

(1, 1)
░░░░░
░O═╗░
░║░║░
░╚═╝░
░░░░░



4

In [76]:
test_inputs_b = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ"""

test_runner_b = MapPositionRunner(test_inputs_b)
print(test_runner_b)
print(test_runner_b.starting_position.coords)
test_distance_b = test_runner_b.travel_loop()
print(test_runner_b.traveled_loop)
test_distance_b

╗═╔╗═
░╔╝║╗
O╝╚╚╗
║╔══╝
╚╝░╚╝

(2, 0)
╗═╔╗═
░╔╝║╗
O╝╚╚╗
║╔══╝
╚╝░╚╝



8

In [77]:
inputs = open("../inputs/10.txt").read()

runner = MapPositionRunner(inputs)
# print(runner)
print(runner.starting_position.coords)
distance = runner.travel_loop()
print(runner.traveled_loop)
distance

(58, 51)
╔░╔╗╗░╔╗╔╗╔╔╝╝░░╗╗╔╔░══╚╗╗╗░═╗╗╗╗══╔╗╔╗╔╝╔╔╔╗╔╔║░╔╗╗╔╔║═╗═║═╔╗═╗═╔╔║╔╝═╔╔╝╗═╔══╔║═╚══╗╚║═╗╔╔╔╔═══╗═╔╝░╗╝░╔═╗╝░╔║╗═╚╔╗╚╗║░╔═╚║╗░░░╚╔══╚╚╗╔═╗╗
║╗╗╔╚═╔╗╔═╔╝╝╔╗╔╝═╔╝╔░╚║║╔╚╝╔╗╝╔╝╗╔╝╗╗╚╝═╗═║║╚╗╗═╔═╗║║╚╗║╚╚░╝╝░╚═╔╔╝═║░╔╝╝║╔║╔╝╝║░░║═╗░╗╗╚╚╝╔╗╝╝═║╗╚╗╗╝║░║╚║═╚╗║╚═║╚╚═╚╗═║╗░╚╗╔╗═╔╚╝║╔╔╚╝═╚╝
╝╔╔╝╝╔╔═║╝░║╚╗░╗╔╚║╗║╗░╚╚═╝░═╝╔║║╝║═╚║╗║╔╚╗╚╗╔╝║░══╗║╔═══░╚═╝░║═╗╝╚═╔╗═╗══║╗╚╗░╗░╔░═╝╚╗╚║╝░╔═╝╝╝╗╚╔╝╝╚╚╗╔║═╗░╚╔═╗░╔╗╚╔║╚═╚╔╚╝╚═╗╔╝╚║═║╗║╗░╔╝
╚╚╝╗╚╝╚╗║║╔╔═║═╝╗╝║║══╗╝╗░╔╝╚╗═░╔═╝░║░░╔╗╚║╔╔║╝╗╗╚╔╔╚╔╝╚║╔╗║╔╚╝╚╔╔╝═╝╗░╚╝║╚║╔╝═╔╗╝░╚╔║══╝╔══╗║╚║║╗░║╝╝║╚╔╝╔╗╝═╗░║░║╝═║═╝╔╔╝║╝╗║╚║══╔░══╔╗╗░╗
╚╚╚╝╚║═╗═╗═║═╔╗═╗╗░╝╝░║═╔═╗╝║║╚═╔╗░║░╗═╚║╗╝╔╚╝╚╝║░╗══║╗╝╚░╔╝╝╝═╔░╚░╗╔═╗╝╔╗╝║╝╚║╚╝═░░╔╗║╗░╔═╚╚═╔╔═║╗╝║╔╚╝║░╝░╔╔╔╗╔╗╝╝╚╔░║╔╗╚╚░╔║╗╚╗═║╗╝╝╝╚╔░╚
╔║╔║╔║║╚░║╚╗║║║╔╝║╗╚══╝╔╚╗╚═║═╗░╗══╝╝║╚╚╝╗╚╔╝░╚╚║░╚╝╔╚╚╗░╔░║╗╔═╚╝╝═╝╚╚╝░║╗╝║╚╚╔╗╝░║╗║╚═╗╗║═╝╚╔╗╝╗╚╝╔═╗╗╔║░║║╗░╔╗║║░║░╚═║═║░╔╔╔╝╝╗╝╔╚╔╗═╔║╝░║
═╗╗░║╝╗║═╗░╚═╝╗╚═╔╗╗╚║╝╗╗╚╔╚╚║║╔║╗╔╔╗║░░║╝░╚═╚═░╚╗╚║╚╗╔╚╗╗░═╗╚╗╝║║░╔═╚╝═╝╝░║╗║║╚═░╔╔╚═╗╚╗╝░╝╝╚╚╗╔╗╗╔═╚╗╗║══╚═╔║╚╝║═╗╗║╗║╝║═╗═╚╚░║╚╚╗║║╔═╝╔╚═
╚╗══

6942

## Part 2

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

```
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
```
The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```
In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```
In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```
The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```
In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```
Here are just the tiles that are enclosed by the loop marked with I:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```
In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?